# 1.环境配置

In [ ]:
#安装相应环境
!pip install d2l
!pip install matplotlib

In [ ]:
#导入相应库
%matplotlib inline
import torch
from d2l import torch as d2l
import numpy as np
import pandas as pd
from torch import functional as F
from torch import nn

# 2.数据预处理 

In [ ]:
test_data = pd.read_csv(r'../input/zerodata/used_car_testB_20200421.csv',sep=' ')
test_data = test_data.drop(['SaleID'],axis=1)
test_data = test_data.drop(['regionCode'],axis=1)
test_data = test_data.drop(['name'],axis=1)

l = []
for i in range(test_data['regDate'].shape[0]):
    temp_str = str(test_data['regDate'][i])
    year = int(temp_str[:4])
    month = int(temp_str[4:6])
    l.append(round(2022-year+month/12,2))

l1 = []
for i in range(test_data['creatDate'].shape[0]):
    temp_str = str(test_data['creatDate'][i])
    year = int(temp_str[:4])
    month = int(temp_str[4:6])
    l1.append(round(2022-year+month/12,2))
    
test_data['notRepairedDamage'].replace('-',np.nan,inplace=True)
test_data['regDate'] = pd.Series(l)
test_data['creatDate'] = pd.Series(l1)
test_data['gearbox'] = test_data['gearbox'].astype(object)
test_data['notRepairedDamage'] = test_data['notRepairedDamage'].astype(object)
test_data['seller'] = test_data['seller'].astype(object)
test_data['offerType'] = test_data['offerType'].astype(object)
#再进行数据的清洗，将Nan数据用数据中的平均值代替,并进行数据的归一化
# 若无法获得测试数据，则可根据训练数据计算均值和标准差
all_features2 = test_data
numeric_features2 = all_features2.dtypes[all_features2.dtypes != 'object'].index
all_features2[numeric_features2] = all_features2[numeric_features2].apply(lambda x: (x - x.mean()) / (x.std()))
# 在标准化数据之后，所有均值消失，因此我们可以将缺失值设置为均值
all_features2[numeric_features2] = all_features2[numeric_features2].fillna(all_features2[numeric_features2].mean())

#对非数字的特征进行独热编码
# “Dummy_na=True”将“na”（缺失值）视为有效的特征值，并为其创建指示符特征
all_features2 = pd.get_dummies(all_features2, dummy_na=True)
all_features2.shape #可以看到，进行独热编码后，总共有49个特征

n_test = test_data.shape[0]
test_features = torch.tensor(all_features2[:n_test].values, dtype=torch.float32)

train_data = train_data.drop(['SaleID'],axis=1)
true_price = torch.tensor(train_data['price'],dtype=torch.float32)
train_data = train_data.drop(['price'],axis=1)
train_data = train_data.drop(['regionCode'],axis=1)
train_data = train_data.drop(['name'],axis=1)

l = []
for i in range(train_data['regDate'].shape[0]):
    temp_str = str(train_data['regDate'][i])
    year = int(temp_str[:4])
    month = int(temp_str[4:6])
    l.append(round(2022-year+month/12,2))

l1 = []
for i in range(train_data['creatDate'].shape[0]):
    temp_str = str(train_data['creatDate'][i])
    year = int(temp_str[:4])
    month = int(temp_str[4:6])
    l1.append(round(2022-year+month/12,2))
    
train_data['notRepairedDamage'].replace('-',np.nan,inplace=True)
train_data['regDate'] = pd.Series(l)
train_data['creatDate'] = pd.Series(l1)
train_data['gearbox'] = train_data['gearbox'].astype(object)
train_data['notRepairedDamage'] = train_data['notRepairedDamage'].astype(object)
train_data['seller'] = train_data['seller'].astype(object)
train_data['offerType'] = train_data['offerType'].astype(object)

#训练集同理
all_features = train_data
numeric_features = all_features.dtypes[all_features.dtypes != 'object'].index
str_features = all_features.dtypes[all_features.dtypes == 'object'].index
all_features[numeric_features] = all_features[numeric_features].apply(lambda x: (x - x.mean()) / (x.std()))
# 在标准化数据之后，所有均值消失，因此我们可以将缺失值设置为均值
all_features[numeric_features] = all_features[numeric_features].fillna(all_features[numeric_features].mean())

#对非数字的特征进行独热编码
# “Dummy_na=True”将“na”（缺失值）视为有效的特征值，并为其创建指示符特征
all_features = pd.get_dummies(all_features, dummy_na=True)
all_features.shape #可以看到，进行独热编码后，总共有34个特征

#将dataframe转换为tensor便于后续的训练
n_train = train_data.shape[0]
train_features = torch.tensor(all_features[:n_train].values, dtype=torch.float32) 
train_labels = true_price.reshape(-1,1)[:n_train]

# 3.网络的定义

In [ ]:
#定义交叉熵损失
loss = nn.L1Loss() #回归问题，使用nn.L1loss()
in_features = train_features.shape[1]
#定义神经网络
# 构造网络模型
loss = nn.MSELoss()
in_features = train_features.shape[1]

# 构造网络模型
class model(torch.nn.Module):
    def __init__(self):
        super(model, self).__init__()
        self.linear1 = torch.nn.Linear(in_features,24)
        self.linear2 = torch.nn.Linear(24,1)
        self.relu = torch.nn.ReLU()
        self.dropout1 = torch.nn.Dropout(0.5)

    def forward(self, x):
        x = self.relu(self.linear1(x))
        x = self.dropout1(x)
        x = self.linear2(x)
        return x

net = model()

def log_rmse(net, features, labels):
    # 为了在取对数时进一步稳定该值，将小于1的值设置为1
    clipped_preds = torch.clamp(net(features), 1, float('inf'))
    rmse = torch.sqrt(loss(torch.log(clipped_preds),
                           torch.log(labels)))
    return rmse.item()

# 4.训练和交叉验证

In [ ]:
def train(train_features, train_labels, test_features, test_labels,
          num_epochs, learning_rate, weight_decay, batch_size):
    train_ls, test_ls = [], []
    train_iter = d2l.load_array((train_features, train_labels), batch_size)
    # 这里使用的是Adam优化算法
    optimizer = torch.optim.Adam(net.parameters(),lr = learning_rate,weight_decay = weight_decay)
    for epoch in range(num_epochs):
        for X, y in train_iter:
            optimizer.zero_grad()
            l = loss(net(X), y)
            l.backward()
            optimizer.step()
        train_ls.append(log_rmse(net,train_features,train_labels))
        if test_labels is not None:
            test_ls.append(log_rmse(net,test_features,test_labels))
    return train_ls, test_ls

def get_k_fold_data(k, i, X, y):
    assert k > 1
    fold_size = X.shape[0] // k
    X_train, y_train = None, None
    for j in range(k):
        idx = slice(j * fold_size, (j + 1) * fold_size)
        X_part, y_part = X[idx, :], y[idx]
        if j == i:
            X_valid, y_valid = X_part, y_part
        elif X_train is None:
            X_train, y_train = X_part, y_part
        else:
            X_train = torch.cat([X_train, X_part], 0)
            y_train = torch.cat([y_train, y_part], 0)
    return X_train, y_train, X_valid, y_valid

def k_fold(k, X_train, y_train, num_epochs, learning_rate, weight_decay,
           batch_size):
    train_l_sum, valid_l_sum = 0, 0
    for i in range(k):
        data = get_k_fold_data(k, i, X_train, y_train)
        train_ls, valid_ls = train(*data, num_epochs, learning_rate,
                                   weight_decay, batch_size)
        train_l_sum += train_ls[-1]
        valid_l_sum += valid_ls[-1]
        if i == 0:
            d2l.plot(list(range(1, num_epochs + 1)), [train_ls, valid_ls],
                     xlabel='epoch', ylabel='rmse', xlim=[1, num_epochs],
                     legend=['train', 'valid'], yscale='log')
        print(f'折{i + 1}，训练log rmse{float(train_ls[-1]):f}, '
              f'验证log rmse{float(valid_ls[-1]):f}')
    return train_l_sum / k, valid_l_sum / k

k, num_epochs, lr,weight_decay,batch_size = 5,600,0.002,0.00001,256
train_l, valid_l = k_fold(k, train_features, train_labels, num_epochs, lr,weight_decay,batch_size)
print(f'{k}-折验证: 平均训练log rmse: {float(train_l):f}, '
      f'平均验证log rmse: {float(valid_l):f}')
torch.save(net, 'net.pth') #模型保存

# 5.训练结果
## <img src= "image-20220804110342981.png" style="zoom:70%" />